## Bridges EDA

In [1]:
import pandas as pd
import math

In [2]:
# Read the Bridges file
bridges = pd.read_excel("../data/BMMS_overview.xlsx")

In [33]:
# Keep only bridges on the N1 road
bridges = bridges.loc[bridges["road"] == "N1"]

# Define bright length bins and labels
length_bins = [0, 10, 50, 200, math.inf]
length_labels = ['S', 'M', 'L', 'XL']

# Categorize bridges per length class
bridges["length_class"] = pd.cut(bridges["length"], bins=length_bins, include_lowest=False, right=False, labels=length_labels)

keep_columns = ['km', 'type', 'LRPName', 'name', 'length', 'condition', 'chainage', 'lat', 'lon', 'length_class']
bridges_clean = bridges[keep_columns]
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
for val in bridges_clean['LRPName']:
    # Check if all last items are letters
#     if val[-1] not in letters:
#         print(val[-1])

,km,type,LRPName,name,length,condition,chainage,lat,lon,length_class
0,1.800,Box Culvert,LRP001a,.,11.30,A,1.800,23.698739,90.458861,M
1,4.925,Box Culvert,LRP004b,.,6.60,A,4.925,23.694664,90.487775,S
2,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,8.976,23.705060,90.523214,XL
3,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,10.880,23.694391,90.537574,S
4,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,10.897,23.694302,90.537707,S
...,...,...,...,...,...,...,...,...,...,...
19378,426.915,RCC Girder Bridge,LRP423b,Balu Khali Bridge,7.90,D,426.915,21.129768,92.187047,S
19379,427.622,RCC Girder Bridge,LRP424a,Wheke Kang Bridge,6.40,D,427.622,21.125104,92.191367,S
19380,429.081,RCC Girder Bridge,LRP425c,Tasse Bridge,12.80,D,429.081,21.114558,92.198021,M
19381,436.789,RCC Girder Bridge,LRP433b,Naya Bazar(2) Bridge,9.40,D,436.789,21.056294,92.226645,S
